In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np

from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, Flatten, Dense, AvgPool2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.applications.resnet import ResNet50

In [ ]:
def load_train(path):
    # все уже выложено на сервер в какую-то директорию path с поэтому для получения файла labels в коде функции нужно указать путь
    labels = pd.read_csv(path + 'labels.csv')
    # Из фотографий в папках класс ImageDataGenerator формирует батчи с изображениями и метками классов
    train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25, horizontal_flip=True)
    train_datagen_flow = train_datagen.flow_from_dataframe( # «поток из директории»
    dataframe=labels,
    directory=path + 'final_files/', # папка, в которой хранится датасет
    x_col='file_name',
    y_col='real_age',
    target_size=(224, 224), # к какому размеру приводить изображения
    class_mode='raw', # в каком виде выдавать метки классов
    batch_size=16, # размер батча
    subset='training', # как называется выборка
    seed=12345) # фиксируем генератор случайных чисел

    return train_datagen_flow

In [ ]:
def load_test(path):
    labels = pd.read_csv(path + 'labels.csv')
    test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)
    test_datagen_flow = test_datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path + 'final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=16,
        class_mode='raw',
        subset='validation',
        seed=12345)

    return test_datagen_flow


In [ ]:
def create_model(input_shape):
    backbone = ResNet50(input_shape=input_shape,
                        #classes=1000,
                        weights='imagenet',
                        include_top=False)

    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(12, activation='relu'))

    optimizer = Adam(lr=0.0001)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

    return model

In [ ]:
def train_model(model, train_data, test_data, batch_size=None, epochs=10,
               steps_per_epoch=None, validation_steps=None):

    model.fit(train_data,
              validation_data=test_data,
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, shuffle=True)

    return model